# Part 1

In [1]:
import pandas as pd

!pip install datasets
from datasets import Dataset



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
from datasets import load_dataset
data = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
df = pd.DataFrame(data['train'])

from sklearn.model_selection import train_test_split

df_sampled = df.sample(n=1000, random_state=42).reset_index(drop=True)

# Split into train (80%), test (10%), and validation (10%)
train_df, temp_df = train_test_split(df_sampled, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


                                                  text  label
29   The only thing I remember about this movie are...      0
535  Ned Kelly (Ledger), the infamous Australian ou...      0
695  "Hot Millions" is a well-written, well-acted t...      1
557  It must have been several years after it was r...      1
836  Dick Clement and Ian La Frenais have a solid h...      1


In [14]:
!pip install transformers

In [15]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [16]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

# Tokenize the datasets
train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized = val_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns (keeping only tokenized inputs & labels)
train_tokenized = train_tokenized.remove_columns(["text"])
val_tokenized = val_tokenized.remove_columns(["text"])
test_tokenized = test_tokenized.remove_columns(["text"])

# Convert dataset format to PyTorch tensors
train_tokenized.set_format("torch")
val_tokenized.set_format("torch")
test_tokenized.set_format("torch")

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [22]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [29]:
import torch
import evaluate
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support # Import accuracy_score and precision_recall_fscore_support


accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [24]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
training_args = TrainingArguments(
    run_name="imdb_sentiment",
    output_dir='./distilbert-imdb',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    load_best_model_at_end=True,
)

# we setup the trainer with all our previous configurations
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-30-4b86987adee2>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.783117,0.810000,0.777778,0.857143,0.815534
2,No log,0.825488,0.820000,0.754098,0.938776,0.836364


TrainOutput(global_step=100, training_loss=0.11346127510070801, metrics={'train_runtime': 2539.811, 'train_samples_per_second': 0.63, 'train_steps_per_second': 0.039, 'total_flos': 105973918924800.0, 'train_loss': 0.11346127510070801, 'epoch': 2.0})

In [31]:
trainer.evaluate(test_tokenized)

{'eval_loss': 0.7115501165390015,
 'eval_accuracy': 0.83,
 'eval_precision': 0.8333333333333334,
 'eval_recall': 0.8490566037735849,
 'eval_f1': 0.8411214953271028,
 'eval_runtime': 46.2233,
 'eval_samples_per_second': 2.163,
 'eval_steps_per_second': 0.151,
 'epoch': 2.0}

In [33]:
model.save_pretrained("./distilbert_sentiment_model")
tokenizer.save_pretrained("./distilbert_sentiment_model")

('./distilbert_sentiment_model/tokenizer_config.json',
 './distilbert_sentiment_model/special_tokens_map.json',
 './distilbert_sentiment_model/vocab.txt',
 './distilbert_sentiment_model/added_tokens.json')

In [34]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="./distilbert_sentiment_model", tokenizer="./distilbert_sentiment_model")
print(classifier(["I love this movie!", "This was a terrible experience."]))

Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9984204769134521}, {'label': 'LABEL_0', 'score': 0.9920092821121216}]


In [35]:
!pip install huggingface_hub

In [36]:
from huggingface_hub import notebook_login
notebook_login()

In [39]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load model and tokenizer from saved directory
model = AutoModelForSequenceClassification.from_pretrained("./distilbert_sentiment_model")
tokenizer = AutoTokenizer.from_pretrained("./distilbert_sentiment_model")

# Push to Hugging Face Hub
model.push_to_hub("nutukoira/distilbert_sentiment_model")
tokenizer.push_to_hub("nutukoira/distilbert_sentiment_model")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nutukoira/distilbert_sentiment_model/commit/2b544341bfff82ca1d566ba6430d2916cb1bc70e', commit_message='Upload tokenizer', commit_description='', oid='2b544341bfff82ca1d566ba6430d2916cb1bc70e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nutukoira/distilbert_sentiment_model', endpoint='https://huggingface.co', repo_type='model', repo_id='nutukoira/distilbert_sentiment_model'), pr_revision=None, pr_num=None)

In [40]:
model_link = "https://huggingface.co/nutukoira/distilbert_sentiment_model"
print(f"✅ Model successfully uploaded! You can view it here: {model_link}")

✅ Model successfully uploaded! You can view it here: https://huggingface.co/nutukoira/distilbert_sentiment_model


Github repo: https://github.com/nuutti14/sentiment-ui